## NeuroDet version 0

### Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import pathlib
import math


# Disabling GPU for the moment because of the lack of the memory
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Loading the dataset of Brain Scan images
Source of the Dataset: [Kaggle-Brain Tumor Classification](https://www.kaggle.com/sartajbhuvaji/brain-tumor-classification-mri?select=Testing)<br>
Reference for operations performed : [Tensorflow tutorial: Load Images](https://www.tensorflow.org/tutorials/load_data/images)

In [2]:
categories_path = {'glioma_tumor': '/glioma_tumor', 'meningioma_tumor': '/meningioma_tumor', 
                   'pituitary_tumor': '/pituitary_tumor', 'no_tumor':'/no_tumor'}
train_path = 'BrainMRI/Training'
test_path = 'BrainMRI/Testing'

#train_glioma_dir = pathlib.Path(train_path + categories_path['glioma_tumor'])
train_dir = pathlib.Path(train_path)
test_dir = pathlib.Path(test_path)

# Defining the parameters of the dataset
batch_size = 32 #was 128 reduce to 16 to fit the data on the gpu ram
img_height = 256 # was 128
img_width = 256 # was 128

# Loading the train dataset using keras.utils.image_dataset_from_directory
# To use this method, please ensure you have tf.nigthly installed 
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size, color_mode = 'grayscale')

test_ds = tf.keras.utils.image_dataset_from_directory(test_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size, color_mode = 'grayscale')

print(train_ds.class_names)
print(test_ds.class_names)

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']


In [3]:
augment_layer = tf.keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=1474)
train_ds = train_ds.map(
  lambda x, y: (augment_layer(x), y))
print(len(train_ds))

90


In [ ]:
# Looking at an instance of pituitary tumor from the dataset
PIL.Image.open(list(train_dir.glob('pituitary_tumor/*.jpg'))[1])

### Visualizing the train dataset

In [ ]:
# added np.squeeze to fix the dimension error on plt.imshow.
# added cmap = 'gray' to specify that we are displaying grayscale images
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.squeeze(images[i].numpy().astype("uint8")),cmap='gray')
        plt.title( train_ds.class_names[labels[i]])
        plt.axis("off")

### Normilizing the training and testing set

In [4]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))
print(len(train_ds))

90


### Implementing the CNN (based on ResNet)
[tf.keras.layers.MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)<br>
[tf.keras.layers.Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)<br>
[Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf)

## Version 1 CNN

In [4]:
# Version 1 acheive 72% for 5 epoch --> 75% acc after 15 epoch
class CNNResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(CNNResNet, self).__init__()
        
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
        self.norm = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max1 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        
        self.conv2 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, activation='relu', padding ="same")
        self.max2 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.conv3 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, activation='relu', padding ="same")
        self.max3 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.flat = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(80, activation='relu')
        self.out = tf.keras.layers.Dense(num_classes, activation = 'softmax')
        
    def call(self, batch_input):
        c1 = self.conv1(batch_input)
        c1 = tf.nn.relu(self.norm(c1))
        m1 = self.max1(c1)
        
        #print("m_1 shape")
        #print(m1.shape)
        
        c2 = self.conv2(m1)
        m2 = self.max2(c2)
        
        #print("m_2 shape")
        #print(m2.shape)
        
        c3 = self.conv2(m2)
        m3 = self.max2(c3)
        
        #print("m_3 shape")
        #print(m3.shape)
        
        # residual add
        #resNet_add = batch_input + m3
        
        flat = self.flat(m3)
        dense = self.dense(flat)
        
        return self.out(dense)
    
    def loss(self, probas, labels):
        return tf.reduce_mean(tf.keras.metrics.sparse_categorical_crossentropy(labels, probas))
    
    def train(self, train_dataset):
        list_loss = []
        for images, labels in  train_dataset:
            with tf.GradientTape() as tape:
                probas = self.call(images)
                loss = self.loss(probas, labels)
                acc = self.accuracy(probas, labels)
                #print("Loss for the batch {}, accuracy {}".format(loss, acc))
                list_loss.append(loss)
                
            gradients = tape.gradient(loss, self.trainable_variables)
            self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        print("avg Loss for the epoch {}".format(np.average(np.array(list_loss))))
        

    def test(self, test_dataset):
        # maybe return all information to also output roc plots?
        accs = []
        for images, labels in test_dataset:
            probas = self.call(images)
            acc = self.accuracy(probas, labels)
            accs.append(acc)
            
        return accs
        
    
    def accuracy(self, probas, labels):
        correct_predictions = tf.equal(tf.argmax(probas, 1), tf.cast(labels, tf.int64))
        return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [5]:
cnn_resNet = CNNResNet(4)

for i in range(30):
    print("epoch "+str(i))
    cnn_resNet.train(train_ds)

epoch 0
avg Loss for the epoch 0.9071904420852661
epoch 1
avg Loss for the epoch 0.48034706711769104
epoch 2
avg Loss for the epoch 0.30205032229423523
epoch 3
avg Loss for the epoch 0.1745443493127823
epoch 4
avg Loss for the epoch 0.10384365916252136
epoch 5
avg Loss for the epoch 0.07792734354734421
epoch 6
avg Loss for the epoch 0.02892071194946766
epoch 7
avg Loss for the epoch 0.027329551056027412
epoch 8
avg Loss for the epoch 0.013600606471300125
epoch 9
avg Loss for the epoch 0.06659140437841415
epoch 10
avg Loss for the epoch 0.10481758415699005
epoch 11
avg Loss for the epoch 0.019645199179649353
epoch 12
avg Loss for the epoch 0.0020508889574557543
epoch 13
avg Loss for the epoch 0.0006573863211087883
epoch 14
avg Loss for the epoch 0.00040436495328322053
epoch 15
avg Loss for the epoch 0.00011085155711043626
epoch 16
avg Loss for the epoch 8.348435221705586e-05
epoch 17
avg Loss for the epoch 6.727407890139148e-05
epoch 18
avg Loss for the epoch 5.794040407636203e-05
epoch

In [6]:
print(tf.reduce_mean(cnn_resNet.test(test_ds)))

tf.Tensor(0.73221153, shape=(), dtype=float32)


## Version 2 CNN

In [ ]:
# Version 2: Achieve 75% accuracy on testing set for 20 epoch
class CNNResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(CNNResNet, self).__init__()
        
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
        
         #previous kernel filter number  that produced .75 was 10. 16 filter is not good
        self.norm = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.conv2 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max1 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        
        self.conv3 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, activation='relu', padding ="same")
        self.conv4 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, activation='relu', padding ="same")
        self.max2 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.conv5 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, activation='relu', padding ="same")
        self.max3 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.flat = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(80, activation='relu')
        self.out = tf.keras.layers.Dense(num_classes, activation = 'softmax')
        
    def call(self, batch_input):
        c1 = self.conv1(batch_input)
        c2 = self.conv2(c1)
        c2 = tf.nn.relu(self.norm(c2))
        m1 = self.max1(c2)
        
        #print("m_1 shape")
        #print(m1.shape)
        
        c3 = self.conv3(m1)
        c4 = self.conv3(c3)
        m2 = self.max2(c4)
        
        #print("m_2 shape")
        #print(m2.shape)
        
        c5 = self.conv4(m2)
        m3 = self.max2(c5)
        
        #print("m_3 shape")
        #print(m3.shape)
        
        # residual add
        #resNet_add = batch_input + m3
        
        flat = self.flat(m3)
        dense = self.dense(flat)
        
        return self.out(dense)
    
    def loss(self, probas, labels):
        return tf.reduce_mean(tf.keras.metrics.sparse_categorical_crossentropy(labels, probas))
    
    def train(self, train_dataset):
        list_loss = []
        for images, labels in  train_dataset:
            with tf.GradientTape() as tape:
                probas = self.call(images)
                loss = self.loss(probas, labels)
                acc = self.accuracy(probas, labels)
                #print("Loss for the batch {}, accuracy {}".format(loss, acc))
                list_loss.append(loss)
                
            gradients = tape.gradient(loss, self.trainable_variables)
            self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        print("avg Loss for the epoch {}".format(np.average(np.array(list_loss))))
        

    def test(self, test_dataset):
        # maybe return all information to also output roc plots?
        accs = []
        for images, labels in test_dataset:
            probas = self.call(images)
            acc = self.accuracy(probas, labels)
            accs.append(acc)
            
        return accs
        
    
    def accuracy(self, probas, labels):
        correct_predictions = tf.equal(tf.argmax(probas, 1), tf.cast(labels, tf.int64))
        return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [ ]:
cnn_resNet = CNNResNet(4)

for i in range(25):
    print("epoch "+str(i+1))
    cnn_resNet.train(train_ds)

In [ ]:
print(tf.reduce_mean(cnn_resNet.test(test_ds)))

## Version3 CNN

In [4]:
# Version 3: 
class CNNResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(CNNResNet, self).__init__()
        
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        
         #previous kernel filter number  that produced .75 was 10. 16 filter is not good
        self.norm1_1 = tf.keras.layers.BatchNormalization()
        self.norm1_2 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.conv2 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max1 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        
        self.norm2_1 = tf.keras.layers.BatchNormalization()
        self.norm2_2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.conv4 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max2 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.norm3 = tf.keras.layers.BatchNormalization()
        self.conv5 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max3 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.flat = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(80, activation='relu')
        self.out = tf.keras.layers.Dense(num_classes, activation = 'softmax')
        
    def call(self, batch_input, is_training=False):
        c1 = self.conv1(batch_input)
        c1 = tf.nn.relu(self.norm1_1(c1, training=is_training))
        c2 = self.conv2(c1)
        c2 = tf.nn.relu(self.norm1_2(c2, training=is_training))
        m1 = self.max1(c2)
        
        #print("m_1 shape")
        #print(m1.shape)
        
        c3 = self.conv3(m1)
        c3 = tf.nn.relu(self.norm2_1(c3, training=is_training))
        c4 = self.conv3(c3)
        c4 = tf.nn.relu(self.norm2_2(c4, training=is_training))
        m2 = self.max2(c4)
        
        #print("m_2 shape")
        #print(m2.shape)
        
        c5 = self.conv4(m2)
        c5 = tf.nn.relu(self.norm3(c5, training=is_training))
        m3 = self.max2(c5)
        
        #print("m_3 shape")
        #print(m3.shape)
        
        # residual add
        #resNet_add = batch_input + m3
        
        flat = self.flat(m3)
        dense = self.dense(flat)
        
        return self.out(dense)
    
    def loss(self, probas, labels):
        return tf.reduce_mean(tf.keras.metrics.sparse_categorical_crossentropy(labels, probas))
    
    def train(self, train_dataset):
        list_loss = []
        for images, labels in  train_dataset:
            with tf.GradientTape() as tape:
                probas = self.call(images, is_training = True)
                loss = self.loss(probas, labels)
                acc = self.accuracy(probas, labels)
                #print("Loss for the batch {}, accuracy {}".format(loss, acc))
                list_loss.append(loss)
                
            gradients = tape.gradient(loss, self.trainable_variables)
            self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        print("avg Loss for the epoch {}".format(np.average(np.array(list_loss))))
        

    def test(self, test_dataset):
        # maybe return all information to also output roc plots?
        accs = []
        for images, labels in test_dataset:
            probas = self.call(images)
            acc = self.accuracy(probas, labels)
            accs.append(acc)
            
        return accs
        
    
    def accuracy(self, probas, labels):
        correct_predictions = tf.equal(tf.argmax(probas, 1), tf.cast(labels, tf.int64))
        return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [5]:
cnn_resNet = CNNResNet(4)

for i in range(30):
    print("epoch "+str(i+1))
    cnn_resNet.train(train_ds)

epoch 1
avg Loss for the epoch 1.282031774520874
epoch 2
avg Loss for the epoch 0.45946231484413147
epoch 3
avg Loss for the epoch 0.28875723481178284
epoch 4
avg Loss for the epoch 0.22214679419994354
epoch 5
avg Loss for the epoch 0.13820815086364746
epoch 6
avg Loss for the epoch 0.07370144873857498
epoch 7
avg Loss for the epoch 0.047317326068878174
epoch 8
avg Loss for the epoch 0.046539660543203354
epoch 9
avg Loss for the epoch 0.02806546725332737
epoch 10
avg Loss for the epoch 0.016650326550006866
epoch 11
avg Loss for the epoch 0.011769254691898823
epoch 12
avg Loss for the epoch 0.0100216930732131
epoch 13
avg Loss for the epoch 0.006853950675576925
epoch 14
avg Loss for the epoch 0.0040313471108675
epoch 15
avg Loss for the epoch 0.003794342279434204
epoch 16
avg Loss for the epoch 0.003463100641965866
epoch 17
avg Loss for the epoch 0.0033578325528651476
epoch 18
avg Loss for the epoch 0.0020669796504080296
epoch 19
avg Loss for the epoch 0.0018607914680615067
epoch 20
avg

In [6]:
print(tf.reduce_mean(cnn_resNet.test(test_ds)))

tf.Tensor(0.7807692, shape=(), dtype=float32)


In [4]:
# Version 3: 
class CNNResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(CNNResNet, self).__init__()
        
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        
         #previous kernel filter number  that produced .75 was 10. 16 filter is not good
        self.norm1_1 = tf.keras.layers.BatchNormalization()
        self.norm1_2 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.conv2 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max1 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        
        self.norm2_1 = tf.keras.layers.BatchNormalization()
        self.norm2_2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.conv4 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max2 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.norm3 = tf.keras.layers.BatchNormalization()
        #self.conv5 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max3 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.flat = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(80, activation='relu')
        self.out = tf.keras.layers.Dense(num_classes, activation = 'softmax')
        
    def call(self, batch_input, is_training=False):
        c1 = self.conv1(batch_input)
        c1 = tf.nn.relu(self.norm1_1(c1, training=is_training))
        c2 = self.conv2(c1)
        c2 = tf.nn.relu(self.norm1_2(c2, training=is_training))
        m1 = self.max1(c2)
        
        #print("m_1 shape")
        #print(m1.shape)
        
        c3 = self.conv3(m1)
        c3 = tf.nn.relu(self.norm2_1(c3, training=is_training))
        c3 = self.conv3(c3)
        c3 = tf.nn.relu(self.norm2_2(c3, training=is_training))
        m2 = self.max2(c3)
        
        #print("m_2 shape")
        #print(m2.shape)
        
        c4 = self.conv4(m2)
        c4 = tf.nn.relu(self.norm3(c4, training=is_training))
        m3 = self.max2(c4)
        
        #print("m_3 shape")
        #print(m3.shape)
        
        # residual add
        #resNet_add = batch_input + m3
        
        flat = self.flat(m3)
        dense = self.dense(flat)
        
        return self.out(dense)
    
    def loss(self, probas, labels):
        return tf.reduce_mean(tf.keras.metrics.sparse_categorical_crossentropy(labels, probas))
    
    def train(self, train_dataset):
        list_loss = []
        for images, labels in  train_dataset:
            with tf.GradientTape() as tape:
                probas = self.call(images, is_training = True)
                loss = self.loss(probas, labels)
                acc = self.accuracy(probas, labels)
                #print("Loss for the batch {}, accuracy {}".format(loss, acc))
                list_loss.append(loss)
                
            gradients = tape.gradient(loss, self.trainable_variables)
            self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        print("avg Loss for the epoch {}".format(np.average(np.array(list_loss))))
        

    def test(self, test_dataset):
        # maybe return all information to also output roc plots?
        accs = []
        for images, labels in test_dataset:
            probas = self.call(images)
            acc = self.accuracy(probas, labels)
            accs.append(acc)
            
        return accs
        
    
    def accuracy(self, probas, labels):
        correct_predictions = tf.equal(tf.argmax(probas, 1), tf.cast(labels, tf.int64))
        return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [5]:
cnn_resNet = CNNResNet(4)

for i in range(30):
    print("epoch "+str(i+1))
    cnn_resNet.train(train_ds)

epoch 1
avg Loss for the epoch 1.4919942617416382
epoch 2
avg Loss for the epoch 0.5002593994140625
epoch 3
avg Loss for the epoch 0.3394598662853241
epoch 4
avg Loss for the epoch 0.22890014946460724
epoch 5
avg Loss for the epoch 0.16161759197711945
epoch 6
avg Loss for the epoch 0.10571421682834625
epoch 7
avg Loss for the epoch 0.07988719642162323
epoch 8
avg Loss for the epoch 0.058037079870700836
epoch 9
avg Loss for the epoch 0.06113129109144211
epoch 10
avg Loss for the epoch 0.05121428146958351
epoch 11
avg Loss for the epoch 0.022130224853754044
epoch 12
avg Loss for the epoch 0.011321836151182652
epoch 13
avg Loss for the epoch 0.006839027162641287
epoch 14
avg Loss for the epoch 0.0034706571605056524
epoch 15
avg Loss for the epoch 0.0029842592775821686
epoch 16
avg Loss for the epoch 0.002776384586468339
epoch 17
avg Loss for the epoch 0.001994882244616747
epoch 18
avg Loss for the epoch 0.0018058548448607326
epoch 19
avg Loss for the epoch 0.0023045078851282597
epoch 20
a

In [6]:
print(tf.reduce_mean(cnn_resNet.test(test_ds)))

tf.Tensor(0.7711538, shape=(), dtype=float32)


## Version 4 CNN

In [5]:
# Version 4: 
class CNNResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(CNNResNet, self).__init__()
        
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        
         #previous kernel filter number  that produced .75 was 10. 16 filter is not good
        self.norm1_1 = tf.keras.layers.BatchNormalization()
        self.norm1_2 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.conv2 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max1 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        
        self.norm2_1 = tf.keras.layers.BatchNormalization()
        self.norm2_2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.conv4 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max2 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.norm3_1 = tf.keras.layers.BatchNormalization()
        self.norm3_2 = tf.keras.layers.BatchNormalization()
        #self.conv5 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        #self.conv6 = tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding ="same")
        self.max3 = tf.keras.layers.MaxPooling2D(padding ="same")
        
        self.flat = tf.keras.layers.Flatten()
        #self.dense1 = tf.keras.layers.Dense(190)
        #self.drop = tf.keras.layers.Dropout(0.25)
        self.dense2 = tf.keras.layers.Dense(80, activation = 'relu')
        self.out = tf.keras.layers.Dense(num_classes, activation = 'softmax')
        
    def call(self, batch_input, is_training=False):
        c1 = self.conv1(batch_input)
        c1 = tf.nn.relu(self.norm1_1(c1, training=is_training))
        c2 = self.conv2(c1)
        c2 = tf.nn.relu(self.norm1_2(c2, training=is_training))
        m1 = self.max1(c2)
        
        #print("m_1 shape")
        #print(m1.shape)
        
        c3 = self.conv3(m1)
        c3 = tf.nn.relu(self.norm2_1(c3, training=is_training))
        c3 = self.conv3(c3)
        c3 = tf.nn.relu(self.norm2_2(c3, training=is_training))
        m2 = self.max2(c3)
        
        #print("m_2 shape")
        #print(m2.shape)
        
        c4 = self.conv4(m2)
        c4 = tf.nn.relu(self.norm3_1(c4, training=is_training))
        m3 = self.max3(c4)
        
        #print("m_3 shape")
        #print(m3.shape)
        
        # residual add
        #resNet_add = batch_input + m3
        
        flat = self.flat(m3)
        #dense1 = self.dense1(flat)
        #dense1 = self.drop(dense1, training=is_training)
        dense2 = self.dense2(flat)
        
        return self.out(dense2)
    
    def loss(self, probas, labels):
        return tf.reduce_mean(tf.keras.metrics.sparse_categorical_crossentropy(labels, probas))
    
    def train(self, train_dataset):
        list_loss = []
        for images, labels in  train_dataset:
            with tf.GradientTape() as tape:
                probas = self.call(images, is_training = True)
                loss = self.loss(probas, labels)
                acc = self.accuracy(probas, labels)
                #print("Loss for the batch {}, accuracy {}".format(loss, acc))
                list_loss.append(loss)
                
            gradients = tape.gradient(loss, self.trainable_variables)
            self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        print("avg Loss for the epoch {}".format(np.average(np.array(list_loss))))
        

    def test(self, test_dataset):
        # maybe return all information to also output roc plots?
        accs = []
        for images, labels in test_dataset:
            probas = self.call(images)
            acc = self.accuracy(probas, labels)
            accs.append(acc)
            
        return accs
        
    
    def accuracy(self, probas, labels):
        correct_predictions = tf.equal(tf.argmax(probas, 1), tf.cast(labels, tf.int64))
        return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [6]:
cnn_resNet = CNNResNet(4)

for i in range(40):
    print("epoch "+str(i+1))
    cnn_resNet.train(train_ds)

epoch 1
avg Loss for the epoch 1.278624415397644
epoch 2
avg Loss for the epoch 0.762274444103241
epoch 3
avg Loss for the epoch 0.6179163455963135
epoch 4
avg Loss for the epoch 0.5245347023010254
epoch 5
avg Loss for the epoch 0.46221864223480225
epoch 6
avg Loss for the epoch 0.4074830412864685
epoch 7
avg Loss for the epoch 0.37502697110176086
epoch 8
avg Loss for the epoch 0.32681527733802795
epoch 9
avg Loss for the epoch 0.3376762270927429
epoch 10
avg Loss for the epoch 0.2933810353279114
epoch 11
avg Loss for the epoch 0.28921476006507874
epoch 12
avg Loss for the epoch 0.25952664017677307
epoch 13
avg Loss for the epoch 0.25908124446868896
epoch 14
avg Loss for the epoch 0.21469993889331818
epoch 15
avg Loss for the epoch 0.21240085363388062
epoch 16
avg Loss for the epoch 0.2005583643913269
epoch 17
avg Loss for the epoch 0.19345201551914215
epoch 18
avg Loss for the epoch 0.1690511852502823
epoch 19
avg Loss for the epoch 0.18721158802509308
epoch 20
avg Loss for the epoch 

In [7]:
print(tf.reduce_mean(cnn_resNet.test(test_ds)))

tf.Tensor(0.4846154, shape=(), dtype=float32)
